# Final Assignment: Opinion Mining

For this project there are 2 approaches taken: Unsupervised and Supervised and extended further exploration with apriori algorithm. The steps taken are the following: 0) Manually look at data and find inconsistencies such as taggs with { instead of \[ character, or [+] without number 1, or 1 '#' instead of 2. But in the grand scheme of the project these errors are either dealt with a workaround or are insignificant.  1) Set up documents needed and import all libraries needed. 2) Parse the data and plug result into data types with the best ways to be managed, i.e.: dataframes, dicts, and strings. 3)  PreProcess: Preprocess some data to obtain cashedwords. 4) Tokenize: Tokenise the words that are gathered. 5) Aspect Extraction: Retrieve the aspects from the reviews. 6) Opinion mining: Retrieve the top 5 features associated with the product and determine their orientation whether positive or negative. 7) Evaluate algorithm by using metrics.

## Unsupervised learning

1) Import libraries and read the files. Obtain the file directories as well as the product names from the the file names.

In [2]:
import glob
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
# --
import string
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet
# ---
import ast
import collections
from itertools import dropwhile
from nltk.corpus import wordnet as wn
from itertools import product


dirPath = glob.iglob('/Users/*/Downloads/AI Masters/Bath/NLP/Week_8/Data/*')

product_list = []
path_product_list = []

#loop through files and get product names and file directory
for big_file in dirPath:
    files = os.listdir(big_file)
    product_list.append(files)
    for file in files:
        if file.endswith(".txt") and file != "Readme.txt":
            path_product_list.append((os.path.join(big_file, file)))
            with open(os.path.join(big_file, file)) as fp:
                line = fp.readline()

product_list = [i for si in product_list for i in si]
product_list = [x for x in product_list if ".txt" in x and "Readme.txt" not in x]
product_list = [s.replace(".txt", "") for s in product_list]
print(product_list)
print(path_product_list[0][17:])

['Computer', 'Router', 'Speaker', 'Apex AD2600 Progressive-scan DVD player', 'Canon G3', 'Creative Labs Nomad Jukebox Zen Xtra 40GB', 'Nikon coolpix 4300', 'Nokia 6610', 'Canon PowerShot SD500', 'Canon S100', 'Diaper Champ', 'Hitachi router', 'ipod', 'Linksys Router', 'MicroMP3', 'Nokia 6600', 'norton']
/Downloads/AI Masters/Bath/NLP/Week_8/Data\CustomerReviews-3_domains\Computer.txt


Clean the data from Whitespaces, split the features from the text. Insure spaces exist between special characters and the words.

In [3]:
def setup(fileindex):
    text_list = []
    line =''
    #Read lines from file
    with open(path_product_list[fileindex]) as fp:
        lines = list(line for line in (l.strip() for l in fp) if line)
        text_list = lines

    # Create dataframe with splitting sentences at ##
    global df
    df = DataFrame(text_list, columns=['text'])

    fool = lambda x: pd.Series([i for i in reversed(x.split('##'))])
    rev0 = df['text'].apply(fool)
    rev0.rename(columns={0: 'Text', 1: 'Features'}, inplace=True)
    global feature_column
    feature_column = rev0.copy()
    # Create string of sentences without the features
    text_string = df["text"].tolist()
    text_string = ' '.join(text_string)
    text_string = text_string.replace('][', '] [')
    text_string = text_string.replace('[', ' [')

    for i in range(0, len(rev0)):
        if pd.isna(rev0.iloc[i]['Features']):
            rev0.at[i, 'Features'] = rev0.iloc[i]['Text']
            rev0.at[i, 'Text'] = ''
    #split features in dataframe
    foo2 = lambda x: pd.Series([i for i in reversed(x.split(','))])
    rev1 = rev0['Features'].apply(foo2)
    rev1.insert(loc=0, column='Text', value=rev0['Text'])

    for i in range(0, len(rev1.columns) - 1):
        rev1[i] = rev1[i].str.strip()

    for i in range(1, len(rev1.columns)):
        rev1 = rev1.rename(columns={rev1.columns[i]: 'Feature' + str(i)})

    rev1.fillna("", inplace=True)

    rev2 = rev1.iloc[:, 1:len(rev1.columns)].copy()
    for i in range(0, len(rev2.columns)):
        rev2 = rev2.rename(columns={rev2.columns[i]: 'Features_Concat'})

    rev3 = rev2.iloc[:, 0:1].copy()
    for i in range(0, len(rev2.columns) - 1):
        rev3 = rev3.append(rev2.iloc[:, i:i + 1])

    foo3 = lambda x: pd.Series([i for i in x.split('[')])
    rev4 = rev3['Features_Concat'].apply(foo3)
    return [rev1, text_string]

[rev1, text_string] = setup(0)
print(rev1.head())

                                                Text             Feature1  \
0   I purchased this monitor because of budgetary...                        
1   This item was the most inexpensive 17 inch mo...   inexpensive[+1][a]   
2   My overall experience with this monitor was v...          monitor[-1]   
3   When the screen was n't contracting or glitch...  picture quality[-1]   
4   I 've viewed numerous different monitor model...  picture quality[-1]   

      Feature2 Feature3 Feature4 Feature5 Feature6 Feature7 Feature8 Feature9  
0                                                                              
1                                                                              
2                                                                              
3   screen[-1]                                                                 
4  monitor[-1]                                                                 


PreProcess: use nltk to gather stopwords and print first 10. Additional words that can be excluded can be obtained through obtaining high frequency n-grams in all the documents.

In [4]:
def PreProcess():
    cachedStopWords = nltk.corpus.stopwords.words('english')
    cachedStopWords = cachedStopWords + ['im'] + ['i\'m']
    result = (' '.join([word for word in text_string.split() if word.lower() not in cachedStopWords]))
    cStopWords = cachedStopWords
    return cStopWords
cachedStopWords = PreProcess()
print(cachedStopWords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Tokenize: tokenize the words in the list and print first 5. Additional cleaning is performed later depending on need.

In [5]:
def process(text):
    stoplist = set(stopwords.words('english'))  # This is the PreProcessing
    word_list = [word for word in word_tokenize(text.lower()) if
                 word not in stoplist and word not in string.punctuation]  # Tokenization
    return word_list
processed_list = process(text_string)
print(processed_list[:5])

['purchased', 'monitor', 'budgetary', 'concerns', 'inexpensive']


PoS Tagging: Use POS tagging on the full list of words. The reason for that is in order to be able to get opinions, the algorithm will look at the sentence as a whole and determine the orientation based on all the sentence. As well as having a dict for cleaned up words.

In [6]:
def posTagging(rev1):
    df_dict = rev1["Text"].to_dict()
    df_dict_clean_list = list(df_dict.items())
    df_dict_clean_list = [(i, j.replace('[', ' [').replace('],', '] ,').replace('##', '').replace('][', '] [').replace('  ', ' '))
                          for i, j in df_dict_clean_list]
    df_dict_clean_list = [list(elem) for elem in df_dict_clean_list]
    for i in range(0, len(df_dict_clean_list)):
        df_dict_clean_list[i][1] = process(df_dict_clean_list[i][1])
        df_dict_clean_list[i][1] = pos_tag(df_dict_clean_list[i][1])
    df_dict_clean_list = [tuple(l) for l in df_dict_clean_list]
    df_dict_clean_list = dict(df_dict_clean_list)
    print('Clean list:')
    print(list(df_dict_clean_list.items())[:2])

    df_dict = rev1["Text"].to_dict()
    df_dict_list = list(df_dict.items())
    df_dict_list = [(i, j.replace('[', ' [').replace('##', '').replace('][', '] [').replace('  ', ' ')) for i, j in df_dict_list]
    df_dict_list = [list(elem) for elem in df_dict_list]
    for i in range(0, len(df_dict_list)):
        df_dict_list[i][1] = df_dict_list[i][1].split()
        df_dict_list[i][1] = pos_tag(df_dict_list[i][1])
    df_dict_list = [tuple(l) for l in df_dict_list]
    df_dict_list = dict(df_dict_list)
    print('Original List: POS Tagged')
    print(list(df_dict_list.items())[:2])

    return [df_dict_clean_list, df_dict_list]
[df_dict_clean_list, df_dict_list] = posTagging(rev1)

Clean list:
[(0, [('purchased', 'VBN'), ('monitor', 'NN'), ('budgetary', 'JJ'), ('concerns', 'NNS')]), (1, [('item', 'NN'), ('inexpensive', 'JJ'), ('17', 'CD'), ('inch', 'NN'), ('monitor', 'NN'), ('available', 'JJ'), ('time', 'NN'), ('made', 'VBD'), ('purchase', 'NN')])]
Original List: POS Tagged
[(0, [('I', 'PRP'), ('purchased', 'VBD'), ('this', 'DT'), ('monitor', 'NN'), ('because', 'IN'), ('of', 'IN'), ('budgetary', 'JJ'), ('concerns', 'NNS'), ('.', '.')]), (1, [('This', 'DT'), ('item', 'NN'), ('was', 'VBD'), ('the', 'DT'), ('most', 'RBS'), ('inexpensive', 'JJ'), ('17', 'CD'), ('inch', 'NN'), ('monitor', 'NN'), ('available', 'JJ'), ('to', 'TO'), ('me', 'PRP'), ('at', 'IN'), ('the', 'DT'), ('time', 'NN'), ('I', 'PRP'), ('made', 'VBD'), ('the', 'DT'), ('purchase', 'NN'), ('.', '.')])]


Aspect Extraction: In order to gather the aspects, the direction used was to loop through the sentences and words and group any nouns that occur after each other. That way we will be able to capture features such as: "Picture quality", thanks to the tags being 'NN' or 'NNP'. Create 1 list of tuples for feature at sentence index. Create second list of tuples for count of feature in all the sentences.  

In [7]:
def AspectExtraction(df_dict_list):
    prevWord = ''
    prevTag = ''
    currWord = ''
    global aspectListtuple
    aspectListtuple = []
    aspectList = []
    outDict = {}

    # Extracting Aspects
    # Add words together if they are nouns
    for key, value in df_dict_list.items():
        for word, tag in value:
            if (tag == 'NN' or tag == 'NNP'):
                if (prevTag == 'NN' or prevTag == 'NNP'):
                    if (prevWord == word):
                        currWord = word
                    else:
                        currWord = prevWord + ' ' + word
                else:
                    aspectList.append(prevWord.upper())
                    aspectListtuple = aspectListtuple + [[prevWord.upper()] + [key]]
                    currWord = word
            prevWord = currWord
            prevTag = tag

    #Clean list
    for i in range(0,len(aspectListtuple)):
        aspectListtuple[i][0] = [s.replace(" [", "[").replace(" [+1]", "").replace(" [-1]", "").replace("[+1]", "").replace("[-1]", "").replace("[A]", "").replace("[V]", "").replace(
                "[T]", "").replace(",", "") for s in [aspectListtuple[i][0]]][0]
    #Create aspect list tuple of aspect and line it appears in
    aspectListtuple = [tuple(l) for l in aspectListtuple]
    temp = []
    for i in range(0, len(aspectListtuple)):
        if list(aspectListtuple[i])[0] != "":
            temp += [tuple(aspectListtuple[i])]
    aspectListtuple = temp
    temp =[]
    count_dict = collections.Counter([x for (x,y) in aspectListtuple])
    for key, countii in count_dict.items():
        if countii > 1:
            temp += [(key, countii)]
    count_dict = temp
    temp = []
    for aspect,c in aspectListtuple:
        if aspect in [item for sublist in [list(elem) for elem in count_dict] for item in sublist]:
            temp += [(aspect,c)]
    aspectListtuple = temp
    print('First 6 Aspect List tuple by indexed line: ')
    print(aspectListtuple[:6])

    # Clean aspect List
    aspectList2 = []
    for i in range(0, len(aspectList)):
        if ' [' in aspectList[i]:
            temp1 = aspectList[i][aspectList[i].index(" ["):]
            temp2 = aspectList[i][:aspectList[i].index(" [")]
            aspectList2.append(temp1)
            aspectList2.append(temp2)
        else:
            aspectList2.append(aspectList[i])

    aspectList2 = [
        s.replace(" [", "[").replace("[+1]", "").replace("[-1]", "").replace("[A]", "").replace("[V]", "").replace(
            "[T]", "").replace(",", "") for s in aspectList2]
    aspectList2 = list(filter(None, aspectList2))
    aspectList = aspectList2
    # Eliminating aspect which has 1 or less count
    for aspect in aspectList:
        if (aspectList.count(aspect) > 1):
            if (outDict.keys() != aspect):
                outDict[aspect] = aspectList.count(aspect)
    outputAspect = sorted(outDict.items(), key=lambda x: x[1], reverse=True)
    outputAspect = [i for i in outputAspect if i[0] != '##']
    outputAspect = [i for i in outputAspect if len(i[0]) > 1]
    print('Aspects by count are: ')
    print(outputAspect)
    return outputAspect
outputAspect = AspectExtraction(df_dict_list)

First 6 Aspect List tuple by indexed line: 
[('MONITOR', 1), ('ITEM', 1), ('TIME', 1), ('PURCHASE', 2), ('EXPERIENCE', 2), ('MONITOR', 3)]
Aspects by count are: 
[('MONITOR', 72), ('PRICE', 28), ('ACER', 26), ('TIME', 24), ('COMPUTER', 22), ('SCREEN', 17), ('DISPLAY', 14), ('PROBLEM', 14), ('NETBOOK', 13), ('PRODUCT', 12), ('PURCHASE', 9), ('LAPTOP', 9), ('-RRB-', 9), ('CASE', 8), ('LOT', 8), ('ITEM', 7), ('KEYBOARD', 7), ('PICTURE', 7), ('AMAZON', 7), ('LCD', 7), ('MONEY', 7), ('DAY', 6), ('BRAND', 6), ('SIZE', 6), ('-LRB-', 6), ('SOMETHING', 6), ('REPLACEMENT', 6), ('IMAGE', 6), ('QUALITY', 6), ('THING', 6), ('UNIT', 6), ('DEAL', 6), ('SUPPORT', 6), ('EXPERIENCE', 5), ('DELL', 5), ('USER', 5), ('MACHINE', 5), ('CONTRAST', 5), ('BRIGHTNESS', 5), ('RESOLUTION', 5), ('SYSTEM', 5), ('DESK', 5), ('DIFFERENCE', 5), ('APPLE', 5), ('RAM', 4), ('DEVICE', 4), ('OPINION', 4), ('YEAR', 4), ('ANYONE', 4), ('REVIEW', 4), ('COLOR', 4), ('LCD MONITOR', 4), ('LIFE', 4), ('WEIGHT', 4), ('ANYTHING', 4)

Opinion Mining: Determine orientation of word using wordnet.synsets and sentiwordnet.senti_synset. Setup negativeWordSet for which if a sentence starts off positive and later on has a 'but' in the sentence then that will be an indication that it should be now considered negative. Then, setup which sentences at what index are positive or negative based on orientation. Print top 5 features based on count with their associated positive and negative counts. Rather than using the orientation method below a more straightforward way to do it would be to use sentimentanalyser() but this approach was preferred to explore the reasonning behind sentiment.

In [8]:
def orientation(inputWord):
    wordSynset = wordnet.synsets(inputWord)
    if (len(wordSynset) != 0):
        word = wordSynset[0].name()
        orientation = sentiwordnet.senti_synset(word)
        if (orientation.pos_score() > orientation.neg_score()):
            return True
        elif (orientation.pos_score() < orientation.neg_score()):
            return False


def OpinionMining(outputAspect, df_dict_list,n):
    oAspectOpinionTuples = {}
    JOpinionList = []
    orientationCache = {}
    #Switch orientation based on below words
    negativeWordSet = {"don't", "never", "nothing", "nowhere", "noone", "none", "not",
                       "hasn't", "hadn't", "can't", "couldn't", "shouldn't", "won't",
                       "wouldn't", "don't", "doesn't", "didn't", "isn't", "aren't", "ain't"}
    #Determine orientation and switch if applicable
    out = [""]
    for aspect, no in outputAspect:
        aspectTokens = word_tokenize(aspect)
        count = 0
        for key, value in df_dict_list.items():
            condition = True
            isNegativeSen = False
            for subWord in aspectTokens:
                if (subWord in str(value).upper()):
                    condition = condition and True
                else:
                    condition = condition and False
            # Switch orientation if word in negativeWordSet 
            if (condition):
                for negWord in negativeWordSet:
                    if (not isNegativeSen):
                        if negWord.upper() in str(value).upper():
                            isNegativeSen = isNegativeSen or True

                oAspectOpinionTuples.setdefault(aspect, [0, 0, 0])
                #Use words that will make a difference in opinion
                for word, tag in value:

                    if (tag == 'JJ' or tag == 'JJR' or tag == 'JJS' or tag == 'RB' or tag == 'RBR' or tag == 'RBS'):
                        count += 1
                        if (word not in orientationCache):
                            orien = orientation(word)
                            orientationCache[word] = orien
                        else:
                            orien = orientationCache[word]
                        if (isNegativeSen and orien is not None):
                            orien = not orien
                        JOpinionList = JOpinionList + [(key, orien)]
        JOpinionList_ = []
        for i in range(0, len(JOpinionList)):
            if i != len(JOpinionList) - 1:
                if JOpinionList[i][0] != JOpinionList[i + 1][0]:
                    JOpinionList_ = JOpinionList_ + [JOpinionList[i]]
            else:
                JOpinionList_ = JOpinionList_ + [JOpinionList[i]]
        positives = []
        negatives = []
        Neutral = []
        pcount = 0
        necount = 0
        nucount = 0
        # Setup which sentences at what index are positive or negative based on orientation
        for aspect, no in outputAspect:
            for key, Sentiment in JOpinionList_:
                # And JOpinionList_ key is true
                if ' '.join([str(elem) for elem in df_dict_clean_list.get(key)]).find(aspect.lower()) >= 0:
                    if Sentiment == True:
                        pcount += 1
                    if Sentiment == False:
                        necount += 1
                    if Sentiment == None:
                        nucount += 1
            positives = positives + [(aspect, pcount)]
            negatives = negatives + [(aspect, necount)]
            Neutral = Neutral + [(aspect, nucount)]
            pcount = 0
            necount = 0
            nucount = 0
        aspect_reviews = []
        for i in range(0, len(positives)):
            aspect_reviews = aspect_reviews + [list(positives[i]) + [negatives[i][1]] + [Neutral[i][1]]]
        aspect_reviews_c = []
        for i in range(0, len(aspect_reviews)):
            if aspect_reviews[i][0].lower() not in cachedStopWords and len(aspect_reviews[i][0].lower()) > 1:
                aspect_reviews_c = aspect_reviews_c + [aspect_reviews[i]]
        saspect_reviews = sorted(aspect_reviews_c, key=lambda x: x[1], reverse=True)[:5]
        print('Product: ' + str(product_list[n]))
        print('From path: ' + str(path_product_list[n][17:]))
        for i in range(0, len(saspect_reviews)):
            print(saspect_reviews[i][0], ':\t\tPositive ==> ', saspect_reviews[i][1], ':\t\tNegative ==> ',
                  saspect_reviews[i][2])
        return
TopFiveReviews = OpinionMining(outputAspect, df_dict_list,0)

Product: Computer
From path: /Downloads/AI Masters/Bath/NLP/Week_8/Data\CustomerReviews-3_domains\Computer.txt
MONITOR :		Positive ==>  34 :		Negative ==>  12
PRICE :		Positive ==>  6 :		Negative ==>  2
TIME :		Positive ==>  4 :		Negative ==>  0
PURCHASE :		Positive ==>  4 :		Negative ==>  1
PRO :		Positive ==>  4 :		Negative ==>  0


Evaluate metrics of the algorithm by calculating its accuracy. Calculate accuracy solely based on exact word to word comparison.  Then set a threshold for wup_similarity and calculate similarity of related words.

In [22]:
from nltk.corpus import wordnet as wn
from itertools import product
from nltk.corpus import wordnet as wn
from itertools import product
def metrics():
    #Clean up
    remove_from_features = ['[+1]','[-1]','[A]','[V]','[a]','[t]','[v]']
    for i in remove_from_features:
        feature_column['Features'] = feature_column['Features'].str.replace(i, '', regex=False)
    features_at_lines = list(feature_column['Features'].items())
    counti = 0
    temp = []
    features_at_lines = [(sub[1], sub[0]) for sub in features_at_lines]
    for i in features_at_lines:
        if ',' in i[0]:
            spliti = i[0].split(',')
            for ele in spliti:
                temp += [(ele.upper().strip(), i[1])]
        else:
             temp += [(i[0].upper().strip(),i[1])]
    features_at_lines = temp
    #Get features at lines to compare to aspects retrieved at line
    features_at_lines_thathavetags = []
    aspectListtuple_thathavetags = []
    lines_without_tags = []
    for i in features_at_lines:
        if i[0] == '':
            lines_without_tags += [i[1]]
        else:
            features_at_lines_thathavetags += [i]

    for i in aspectListtuple:
        if i[1] not in lines_without_tags:
            aspectListtuple_thathavetags += [i]

    mining_aspects = aspectListtuple_thathavetags
    manual_aspects = features_at_lines_thathavetags
    
    #Calculate word for word accuracy
    true_positive = 0
    for i in manual_aspects:
        if i in mining_aspects:
            true_positive += 1
    #true_positive + false_positive = manual counts
    true_positive_and_false_positive = len(manual_aspects)
    precision = true_positive/true_positive_and_false_positive
    print("Precision word to word is: " + str(precision))
    true_positive_and_false_negative = len(mining_aspects)
    recall1 = true_positive/true_positive_and_false_negative
    print("Recall word to word is " + str(recall1))
    
    #Use wup_similarity for related words accuracy
    true_positive_related1 = 0
    for i in range(0,len(manual_aspects)):
        for ii in range(0,len(mining_aspects)):
            if manual_aspects[i][1] == mining_aspects[ii][1]:
                wordx, wordy = manual_aspects[i][0],mining_aspects[ii][0]
                if wordx == wordy:
                    true_positive_related1 += 1
                else:
                    sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)
                    maxscore = 0
                    for il, j in list(product(*[sem1, sem2])):
                        score = il.wup_similarity(j)
                        if score != None:
                            maxscore = score if maxscore < score else maxscore
                    if maxscore > 0.4:
                        true_positive_related1 += 1
    true_positive_and_false_positive_related1 = len(mining_aspects)
    precision2 = true_positive_related1 / true_positive_and_false_positive_related1
    print("Precision for related words using wup_similarity is: " + str(precision2))
    true_positive_and_false_negative2 = len(manual_aspects)
    recall2 = true_positive_related1/true_positive_and_false_negative2
    print("Recall wup_similarity is " + str(recall2))
metrics = metrics()

Precision word to word is: 0.2909604519774011
Recall word to word is 0.25183374083129584
Precision for related words using wup_similarity is: 0.7799511002444988
Recall wup_similarity is 0.9011299435028248


### Running through all files (Code is copied from above to loop through all files)
#### Cell below is only used to print output
The code below will loop through all the files and grab the features and print the top five features with their positive and negative reviews.

In [10]:
# ---------------------------------Set up Data-------------------
def setup(fileindex):
    text_list = []
    line =''
    with open(path_product_list[fileindex]) as fp:
        lines = list(line for line in (l.strip() for l in fp) if line)
        text_list = lines


    global df
    df = DataFrame(text_list, columns=['text'])

    fool = lambda x: pd.Series([i for i in reversed(x.split('##'))])
    rev0 = df['text'].apply(fool)
    rev0.rename(columns={0: 'Text', 1: 'Features'}, inplace=True)
    global feature_column
    feature_column = rev0.copy()

    text_string = df["text"].tolist()
    text_string = ' '.join(text_string)
    text_string = text_string.replace('][', '] [')
    text_string = text_string.replace('[', ' [')

    for i in range(0, len(rev0)):
        if pd.isna(rev0.iloc[i]['Features']):
            rev0.at[i, 'Features'] = rev0.iloc[i]['Text']
            rev0.at[i, 'Text'] = ''

    foo2 = lambda x: pd.Series([i for i in reversed(x.split(','))])
    rev1 = rev0['Features'].apply(foo2)
    rev1.insert(loc=0, column='Text', value=rev0['Text'])

    for i in range(0, len(rev1.columns) - 1):
        rev1[i] = rev1[i].str.strip()

    for i in range(1, len(rev1.columns)):
        rev1 = rev1.rename(columns={rev1.columns[i]: 'Feature' + str(i)})

    rev1.fillna("", inplace=True)

    rev2 = rev1.iloc[:, 1:len(rev1.columns)].copy()
    for i in range(0, len(rev2.columns)):
        rev2 = rev2.rename(columns={rev2.columns[i]: 'Features_Concat'})

    rev3 = rev2.iloc[:, 0:1].copy()
    for i in range(0, len(rev2.columns) - 1):
        rev3 = rev3.append(rev2.iloc[:, i:i + 1])

    foo3 = lambda x: pd.Series([i for i in x.split('[')])
    rev4 = rev3['Features_Concat'].apply(foo3)
    return [rev1, text_string]


# --------------PreProcessing------------------
def PreProcess():
    cachedStopWords = nltk.corpus.stopwords.words('english')
    cachedStopWords = cachedStopWords + ['im'] + ['i\'m']
    result = (' '.join([WordNetLemmatizer().lemmatize(word.lower()) for word in text_string.split() if word.lower() not in cachedStopWords]))
    cStopWords = cachedStopWords
    return cStopWords


# --------------PreProcessing------------------

# --------------Tokenize------------------
def process(text):
    stoplist = set(stopwords.words('english'))  # This is the PreProcessing
    word_list = [word for word in word_tokenize(text.lower()) if
                 word not in stoplist and word not in string.punctuation]  # Tokenization
    return word_list


# --------------Tokenize------------------

# --------------PoS Tagging------------------
def posTagging(rev1):
    df_dict = rev1["Text"].to_dict()
    df_dict_clean_list = list(df_dict.items())
    df_dict_clean_list = [(i, j.replace('[', ' [').replace('],', '] ,').replace('##', '').replace('][', '] [').replace('  ', ' '))
                          for i, j in df_dict_clean_list]
    df_dict_clean_list = [list(elem) for elem in df_dict_clean_list]
    for i in range(0, len(df_dict_clean_list)):
        df_dict_clean_list[i][1] = process(df_dict_clean_list[i][1])
        df_dict_clean_list[i][1] = pos_tag(df_dict_clean_list[i][1])
    df_dict_clean_list = [tuple(l) for l in df_dict_clean_list]
    df_dict_clean_list = dict(df_dict_clean_list)
    # print(list(df_dict_clean_list.items())[:2])

    df_dict = rev1["Text"].to_dict()
    df_dict_list = list(df_dict.items())
    df_dict_list = [(i, j.replace('[', ' [').replace('##', '').replace('][', '] [').replace('  ', ' ')) for i, j in df_dict_list]
    df_dict_list = [list(elem) for elem in df_dict_list]
    for i in range(0, len(df_dict_list)):
        df_dict_list[i][1] = df_dict_list[i][1].split()
        df_dict_list[i][1] = pos_tag(df_dict_list[i][1])
    df_dict_list = [tuple(l) for l in df_dict_list]
    df_dict_list = dict(df_dict_list)
    # print(list(df_dict_list.items())[:2])

    return [df_dict_clean_list, df_dict_list]


# --------------PoS Tagging------------------

# --------------Aspect Extraction------------------
def AspectExtraction(df_dict_list):
    prevWord = ''
    prevTag = ''
    currWord = ''
    global aspectListtuple
    aspectListtuple = []
    aspectList = []
    outDict = {}

    # Extracting Aspects
    for key, value in df_dict_list.items():
        for word, tag in value:
            if (tag == 'NN' or tag == 'NNP'):
                if (prevTag == 'NN' or prevTag == 'NNP'):
                    if (prevWord == word):
                        currWord = word
                    else:
                        currWord = prevWord + ' ' + word
                        # aspectListtuple = aspectListtuple + [[currWord.upper()] + [key]]
                else:
                    aspectList.append(prevWord.upper())
                    aspectListtuple = aspectListtuple + [[prevWord.upper()] + [key]]
                    currWord = word
            prevWord = currWord
            prevTag = tag

    for i in range(0,len(aspectListtuple)):
        aspectListtuple[i][0] = [s.replace(" [", "[").replace(" [+1]", "").replace(" [-1]", "").replace("[+1]", "").replace("[-1]", "").replace("[A]", "").replace("[V]", "").replace(
                "[T]", "").replace(",", "") for s in [aspectListtuple[i][0]]][0]

    aspectListtuple = [tuple(l) for l in aspectListtuple]
    temp = []
    for i in range(0, len(aspectListtuple)):
        if list(aspectListtuple[i])[0] != "":
            temp += [tuple(aspectListtuple[i])]
    aspectListtuple = temp
    temp =[]
    count_dict = collections.Counter([x for (x,y) in aspectListtuple])
    for key, countii in count_dict.items():
        if countii > 1:
            temp += [(key, countii)]
    count_dict = temp
    temp = []
    for aspect,c in aspectListtuple:
        if aspect in [item for sublist in [list(elem) for elem in count_dict] for item in sublist]:
            temp += [(aspect,c)]
    aspectListtuple = temp
    # print(aspectListtuple)

    # print(aspectList)
    aspectList2 = []
    for i in range(0, len(aspectList)):
        if ' [' in aspectList[i]:
            temp1 = aspectList[i][aspectList[i].index(" ["):]
            temp2 = aspectList[i][:aspectList[i].index(" [")]
            aspectList2.append(temp1)
            aspectList2.append(temp2)
        else:
            aspectList2.append(aspectList[i])

    aspectList2 = [
        s.replace(" [", "[").replace("[+1]", "").replace("[-1]", "").replace("[A]", "").replace("[V]", "").replace(
            "[T]", "").replace(",", "") for s in aspectList2]
    aspectList2 = list(filter(None, aspectList2))
    aspectList = aspectList2
    # Eliminating aspect which has 1 or less count
    for aspect in aspectList:
        if (aspectList.count(aspect) > 1):
            if (outDict.keys() != aspect):
                outDict[aspect] = aspectList.count(aspect)
    outputAspect = sorted(outDict.items(), key=lambda x: x[1], reverse=True)
    outputAspect = [i for i in outputAspect if i[0] != '##']
    outputAspect = [i for i in outputAspect if len(i[0]) > 1]
    
    # print(outputAspect)
    return outputAspect
# --------------Aspect Extraction------------------


# --------------Opinion Mining------------------
def orientation(inputWord):
    wordSynset = wordnet.synsets(inputWord)
    if (len(wordSynset) != 0):
        word = wordSynset[0].name()
        orientation = sentiwordnet.senti_synset(word)
        if (orientation.pos_score() > orientation.neg_score()):
            return True
        elif (orientation.pos_score() < orientation.neg_score()):
            return False


def OpinionMining(outputAspect, df_dict_list,n):
    oAspectOpinionTuples = {}
    JOpinionList = []
    orientationCache = {}
    negativeWordSet = {"don't", "never", "nothing", "nowhere", "noone", "none", "not",
                       "hasn't", "hadn't", "can't", "couldn't", "shouldn't", "won't",
                       "wouldn't", "don't", "doesn't", "didn't", "isn't", "aren't", "ain't"}

    out = [""]
    for aspect, no in outputAspect:
        aspectTokens = word_tokenize(aspect)
        count = 0
        for key, value in df_dict_list.items():
            condition = True
            isNegativeSen = False
            for subWord in aspectTokens:
                if (subWord in str(value).upper()):
                    condition = condition and True
                else:
                    condition = condition and False

            if (condition):
                for negWord in negativeWordSet:
                    if (not isNegativeSen):
                        if negWord.upper() in str(value).upper():
                            isNegativeSen = isNegativeSen or True

                oAspectOpinionTuples.setdefault(aspect, [0, 0, 0])

                for word, tag in value:

                    if (tag == 'JJ' or tag == 'JJR' or tag == 'JJS' or tag == 'RB' or tag == 'RBR' or tag == 'RBS'):
                        count += 1
                        if (word not in orientationCache):
                            orien = orientation(word)
                            orientationCache[word] = orien
                        else:
                            orien = orientationCache[word]
                        if (isNegativeSen and orien is not None):
                            orien = not orien
                        if (orien == True):
                            oAspectOpinionTuples[aspect][0] += 1
                        elif (orien == False):
                            oAspectOpinionTuples[aspect][1] += 1
                        elif (orien is None):
                            oAspectOpinionTuples[aspect][2] += 1
                        
                        JOpinionList = JOpinionList + [(key, orien)]
        JOpinionList_ = []
        for i in range(0, len(JOpinionList)):
            if i != len(JOpinionList) - 1:
                if JOpinionList[i][0] != JOpinionList[i + 1][0]:
                    JOpinionList_ = JOpinionList_ + [JOpinionList[i]]
            else:
                JOpinionList_ = JOpinionList_ + [JOpinionList[i]]
        positives = []
        negatives = []
        Neutral = []
        pcount = 0
        necount = 0
        nucount = 0
        for aspect, no in outputAspect:
            for key, Sentiment in JOpinionList_:
                # And JOpinionList_ key is true
                if ' '.join([str(elem) for elem in df_dict_clean_list.get(key)]).find(aspect.lower()) >= 0:
                    if Sentiment == True:
                        pcount += 1
                    if Sentiment == False:
                        necount += 1
                    if Sentiment == None:
                        nucount += 1
            positives = positives + [(aspect, pcount)]
            negatives = negatives + [(aspect, necount)]
            Neutral = Neutral + [(aspect, nucount)]
            pcount = 0
            necount = 0
            nucount = 0
        aspect_reviews = []
        for i in range(0, len(positives)):
            aspect_reviews = aspect_reviews + [list(positives[i]) + [negatives[i][1]] + [Neutral[i][1]]]
        aspect_reviews_c = []
        for i in range(0, len(aspect_reviews)):
            if aspect_reviews[i][0].lower() not in cachedStopWords and len(aspect_reviews[i][0].lower()) > 1:
                aspect_reviews_c = aspect_reviews_c + [aspect_reviews[i]]
        saspect_reviews = sorted(aspect_reviews_c, key=lambda x: x[1], reverse=True)[:5]
        print('Product: ' + str(product_list[n]))
        print('From path: ' + str(path_product_list[n][17:]))
        for i in range(0, len(saspect_reviews)):
            print(saspect_reviews[i][0], ':\t\tPositive ==> ', saspect_reviews[i][1], ':\t\tNegative ==> ',
                  saspect_reviews[i][2])
        return
# --------------Opinion Mining------------------


for n in range(0, len(path_product_list)):
    [rev1, text_string] = setup(n)
    cachedStopWords = PreProcess()
    processed_list = process(text_string)
    [df_dict_clean_list, df_dict_list] = posTagging(rev1)
    outputAspect = AspectExtraction(df_dict_list)
    TopFiveReviews = OpinionMining(outputAspect, df_dict_list,n)
    # print(TopFiveReviews)
#     if n == 0:
#         metrics = metrics()

Product: Computer
From path: /Downloads/AI Masters/Bath/NLP/Week_8/Data\CustomerReviews-3_domains\Computer.txt
MONITOR :		Positive ==>  34 :		Negative ==>  12
PRICE :		Positive ==>  6 :		Negative ==>  2
TIME :		Positive ==>  4 :		Negative ==>  0
PURCHASE :		Positive ==>  4 :		Negative ==>  1
PRO :		Positive ==>  4 :		Negative ==>  0
Product: Router
From path: /Downloads/AI Masters/Bath/NLP/Week_8/Data\CustomerReviews-3_domains\Router.txt
ROUTER :		Positive ==>  46 :		Negative ==>  24
WORK :		Positive ==>  8 :		Negative ==>  5
ONE :		Positive ==>  8 :		Negative ==>  6
-RRB- :		Positive ==>  6 :		Negative ==>  3
-LRB- :		Positive ==>  6 :		Negative ==>  2
Product: Speaker
From path: /Downloads/AI Masters/Bath/NLP/Week_8/Data\CustomerReviews-3_domains\Speaker.txt
SOUND :		Positive ==>  45 :		Negative ==>  18
SPEAKER :		Positive ==>  13 :		Negative ==>  5
QUALITY :		Positive ==>  9 :		Negative ==>  3
BASS :		Positive ==>  6 :		Negative ==>  0
-RRB- :		Positive ==>  6 :		Negative ==>  2
Pro

## Supervised Learning

After a baseline was established above, we will explore the supervised learning approach. Convert tages to either positive or negative by making use of + and - signs

In [11]:
[rev1, text_string] = setup(0)
from nltk.stem import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

def Supervised():
    df_dict = df["text"].to_dict()
    df_dict_clean_list = list(df_dict.items())
    df_dict_clean_list = [(i, j.replace('[', ' [').replace('],', '] ,').replace('##', '').replace('][', '] [').replace('  ', ' ')) for i, j
                          in df_dict_clean_list]
    df_dict_clean_list = [list(elem) for elem in df_dict_clean_list]
    df_dict_clean_list = [tuple(l) for l in df_dict_clean_list]
    df_dict_clean_list = dict(df_dict_clean_list)
    global df_clean_list
    df_clean_list = pd.DataFrame(df_dict_clean_list.items(), columns=['Index','Text_from_dict'])
    df_clean_list['Plus_counts'] = df_clean_list[['Text_from_dict']].applymap(lambda x: str.count(x, '+'))
    df_clean_list['Minus_counts'] = df_clean_list[['Text_from_dict']].applymap(lambda x: str.count(x, '-'))*-1
    joined_pos_neg = df_clean_list['Plus_counts'] + df_clean_list['Minus_counts']
    df_clean_list['joined_signs'] = joined_pos_neg

    df_dict = df["text"].to_dict()
    df_dict_clean_list = list(df_dict.items())
    df_dict_clean_list = [
        (i, j.replace('[', ' [').replace('],', '] ,').replace('##', '').replace('][', '] [').replace('  ', ' ')) for
        i, j
        in df_dict_clean_list]
    df_dict_clean_list = [list(elem) for elem in df_dict_clean_list]
    return [rev1, text_string, df_clean_list, df_dict_clean_list]
supervised = Supervised()
print(df_clean_list[:5])

   Index                                     Text_from_dict  Plus_counts  \
0      0   I purchased this monitor because of budgetary...            0   
1      1  inexpensive [+1] [a] This item was the most in...            1   
2      2  monitor [-1] My overall experience with this m...            0   
3      3  screen [-1] , picture quality [-1] When the sc...            0   
4      4  monitor [-1] , picture quality [-1] I 've view...            0   

   Minus_counts  joined_signs  
0             0             0  
1             0             1  
2            -1            -1  
3            -2            -2  
4            -2            -2  


Use a different approach to POS tagging and use stemming in addition to the previous lemmatizing and tokenizing done earlier and more ways to clean data. Because it is supervised approach, the data needs to be treated in a different way. I used a similar approach I had taken in a different assignment.

In [12]:
# Describe word
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

stop = stopwords.words('english')

#Clean Data
def clean_preProcess_text(text):
    global clean_dict
    clean_dict = {}
    for i in range(0, len(text)):
        # make text into lower case
        sentence = text.lower()
        # tokenize text & take out punctuations
        sentence = [word_cr.strip(string.punctuation) for word_cr in sentence.split(" ")]
        # Take out any word containing numbers
        sentence = [word_cr for word_cr in sentence if not any(cr.isdigit() for cr in word_cr)]
        # Take out any stop words
        # stop = stopwords.words('english')
        sentence = [x_cr for x_cr in sentence if x_cr not in stop]
        # Take out any blank tokens
        sentence = [t_cr for t_cr in sentence if len(t_cr) > 0]
        # pos_tag sentence
        pos_tags = pos_tag(sentence)
        # Lemmatize sentence this deals with treating the variations of the same word
        sentence = [WordNetLemmatizer().lemmatize(t_cr[0], get_wordnet_pos(t_cr[1])) for t_cr in pos_tags]
        # Take out any word that is just one letter
        sentence = [t_cr for t_cr in sentence if len(t_cr) > 1]
        # Combine everything
        sentence = " ".join(sentence)
        text = sentence
        clean_dict[i] = sentence


#Do POS tagging
def posTagging2(rev1):
    df_dict = rev1["Text"].to_dict()
    df_dict_clean_list = list(df_dict.items())
    df_dict_clean_list = [(i, j.replace('[', ' [').replace('##', '').replace('][', '] [').replace('  ', ' ')) for i, j
                          in df_dict_clean_list]
    df_dict_clean_list = [list(elem) for elem in df_dict_clean_list]
    for i in range(0, len(df_dict_clean_list)):
        df_dict_clean_list[i][1] = process(df_dict_clean_list[i][1])
        df_dict_clean_list[i][1] = pos_tag(df_dict_clean_list[i][1])
    df_dict_clean_list = [tuple(l) for l in df_dict_clean_list]
    df_dict_clean_list = dict(df_dict_clean_list)

    stemmed_dict = {}
    for ins in range(0,len(df_dict_clean_list)):
        temp_str = ''
        for i in range(0, len([list(ele) for ele in list(list(df_dict_clean_list.items())[ins])[1]])):
            temp = [list(ele) for ele in list(list(df_dict_clean_list.items())[ins])[1]]
            temp[i][0] = WordNetLemmatizer().lemmatize(temp[i][0])
            temp[i][0] = PorterStemmer().stem(temp[i][0])
            temp_str = temp_str + " " + temp[i][0]
        stemmed_dict[ins] = temp_str
    # print(list(df_dict_clean_list.items())[:2])
    global df_stem
    df_stem = pd.DataFrame(stemmed_dict.items(), columns=['Index','Stem_Text_from_dict'])
    return [df_dict_clean_list, df_dict_list]

[df_dict_clean_list, df_dict_list] = posTagging2(rev1)

Add additional columns to Features to be able to obtain better predictions. 1) Add SentimentAnalyzer() polarity scores column. 2) Add number of charachters in sentences and number of words. 3) Convert text to vectors. 4) Apply tf-idf. 5) Add sings such as positives and negatives to have as labels. 6) Try different groupings and choose what fits best, Originally grouped labels with above 1 being very positive with label 2 and below 1 being very negative with label -2 but it had lower metrics so went with 3 classes of labels = Neutral, Positive, Negative.

In [13]:
# Clean data
df_clean_list["Text_stem_clean"] = df_stem["Stem_Text_from_dict"]#.apply(lambda x: clean_preProcess_text(x))
# df_dict_list = clean_dict
# First part of appending columns for sentiment analysis
Text_sentiment = SentimentIntensityAnalyzer()
df_clean_list["sentiments"] = df_clean_list["Text_stem_clean"].apply(lambda x: Text_sentiment.polarity_scores(x))
df_clean_list = pd.concat([df_clean_list.drop(['sentiments'], axis=1), df_clean_list['sentiments'].apply(pd.Series)], axis=1)
# append column for # of characters
df_clean_list["nb_chars_cr"] = df_clean_list["Text_from_dict"].apply(lambda x: len(x))
# Append column for # of words
df_clean_list["nb_words_cr"] = df_clean_list["Text_from_dict"].apply(lambda x: len(x.split(" ")))

# Second part of appending columns
# generate the columns for the "doc2vec vector columns"
documents = [TaggedDocument(doc, [i_cd]) for i_cd, doc in enumerate(df_clean_list["Text_stem_clean"].apply(lambda x_cr: x_cr.split(" ")))]
# train the model of Doc2Vec using text data
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
# Convert every document to a vector data
doc2vec_df = df_clean_list["Text_stem_clean"].apply(lambda x_cr: model.infer_vector(x_cr.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x_cr) for x_cr in doc2vec_df.columns]
df_clean_list = pd.concat([df_clean_list, doc2vec_df], axis=1)
# append columns of tf-idfs
tfidf_cd = TfidfVectorizer(min_df=10)
tfidf_result_cd = tfidf_cd.fit_transform(df_clean_list["Text_stem_clean"]).toarray()
tfidf_df = pd.DataFrame(tfidf_result_cd, columns=tfidf_cd.get_feature_names())
tfidf_df.columns = ["word_" + str(x_cd) for x_cd in tfidf_df.columns]
tfidf_df.index = df_clean_list.index
df_clean_list = pd.concat([df_clean_list, tfidf_df], axis=1)
cols = df_clean_list.columns.tolist()
cols = cols[2:4] + cols[6:] + cols[4:5]
df_to_numbers = df_clean_list[cols]

# print(df_clean_list)

df_to_numbers = df_to_numbers.assign(Groups=df_to_numbers['joined_signs'])
print(df_to_numbers.head())
# del df_to_numbers['joined_signs']

   Plus_counts  Minus_counts    neg    neu    pos  compound  nb_chars_cr  \
0            0             0  0.000  1.000  0.000    0.0000           57   
1            1             0  0.000  1.000  0.000    0.0000          121   
2            0            -1  0.508  0.492  0.000   -0.4767           68   
3            0            -2  0.250  0.565  0.185   -0.2023          130   
4            0            -2  0.162  0.838  0.000   -0.4767          190   

   nb_words_cr  doc2vec_vector_0  doc2vec_vector_1  ...  word_turn  word_use  \
0           10          0.113804         -0.032708  ...        0.0       0.0   
1           23          0.009049          0.160494  ...        0.0       0.0   
2           12         -0.042634          0.119056  ...        0.0       0.0   
3           23          0.052556          0.226337  ...        0.0       0.0   
4           35          0.099312          0.251304  ...        0.0       0.0   

    word_ve  word_view  word_want  word_well  word_work  word_

Here is where the classes are determined. Decided to go with 3.

In [14]:
df_to_numbers['Groups'] = np.where((df_to_numbers.joined_signs > 0 ),1, df_to_numbers['Groups'])
df_to_numbers['Groups'] = np.where((df_to_numbers.joined_signs < 0 ),-1, df_to_numbers['Groups'])
print('0 is Neutral', '1 is Positive', '-1 is Negative')
print(df_to_numbers.Groups.value_counts())

0 is Neutral 1 is Positive -1 is Negative
 0    261
 1    151
-1    119
Name: Groups, dtype: int64


Assign labels and features to use in a supervised learning classifier and apply random forest classifier to predict untagged data. Print confusion matrix and classification report to asses the algorithm.

In [15]:
from sklearn.metrics import classification_report
# select the features
label = "Groups"
features = df_to_numbers.columns.tolist()[3:-2]
# Divide the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(df_to_numbers[features], df_to_numbers[label], test_size=0.20,
                                                    random_state=42)


# train a random forest classifier
rf = RandomForestClassifier(n_estimators=10000, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
CM = confusion_matrix(y_test, y_pred)
print("ConfusionM RF:\n", CM)
print(classification_report(y_test, y_pred, target_names=['Neutral', 'Positive', 'Negative']))

ConfusionM RF:
 [[ 9 12  4]
 [ 5 50  1]
 [ 2 12 12]]
              precision    recall  f1-score   support

     Neutral       0.56      0.36      0.44        25
    Positive       0.68      0.89      0.77        56
    Negative       0.71      0.46      0.56        26

    accuracy                           0.66       107
   macro avg       0.65      0.57      0.59       107
weighted avg       0.66      0.66      0.64       107



We can see our f1 score which is the best indicator along the side the confusion matrix shows a value of 69% which should be good enough to predict untagged data.

Predict untagged data with the classifier that was just created

In [16]:
[rev1, text_string] = setup(0)
cachedStopWords = PreProcess()
processed_list = process(text_string)
[df_dict_clean_list, df_dict_list] = posTagging(rev1)
outputAspect = AspectExtraction(df_dict_list)
predicted_y = []
#Predict each untagged sentence
for i in range(0,df_to_numbers[features].shape[0]):
    if df_to_numbers[label].iloc[i] != 0:
        predicted_y += [df_to_numbers[label].iloc[i]]
    else:
        dfi = df_to_numbers[features].iloc[i].reset_index().T
        dfi.columns = list(df_to_numbers[features])
        dfi = dfi.drop(dfi.index[0])
        y_pred_i = rf.predict(dfi)
        predicted_y += [y_pred_i[0]]


list_of_aspects = [i[0] for i in aspectListtuple]
dict_count_of_aspect_positives = {}
dict_count_of_aspect_negatives = {}
for i in list_of_aspects:
    dict_count_of_aspect_positives[i] = 0
    dict_count_of_aspect_negatives[i] = 0
# print(list_of_aspects)
# Add positives and negatives
for aspect,lineindex in aspectListtuple:
    if predicted_y[lineindex] > 0:
        dict_count_of_aspect_positives[aspect] = dict_count_of_aspect_positives[aspect] +1
    if predicted_y[lineindex] < 0:
        dict_count_of_aspect_negatives[aspect] = dict_count_of_aspect_negatives[aspect] + 1

dict_count_of_aspect_positives = {k: v for k, v in sorted(dict_count_of_aspect_positives.items(), key=lambda item: item[1],reverse=True)}
for i in range(0,5):
    print(list(dict_count_of_aspect_positives.items())[i][0], ':\t\tPositive ==> ', list(dict_count_of_aspect_positives.
                items())[i][1], ':\t\tNegative ==> ',list(dict_count_of_aspect_negatives.items())[i][1])
# print(dict_count_of_aspect_positives)

MONITOR :		Positive ==>  32 :		Negative ==>  20
PRICE :		Positive ==>  12 :		Negative ==>  0
COMPUTER :		Positive ==>  9 :		Negative ==>  7
ACER :		Positive ==>  7 :		Negative ==>  1
SCREEN :		Positive ==>  6 :		Negative ==>  3


A comparison can be made between the added sentiments a supervised learning approach can take.

### Additional exploration into Apriori alogrithm
Convert data into dataframe with most frequent n features and then run apriori algorithm to determine rules from the sets of aspects.

In [17]:
from apyori import apriori

apriori_dict ={}
apriori_list = []
index = 0
for i in df_dict_clean_list.items():
    apriori_dict[index] = ' '.join([str(elem) for elem in [at[0] for at in i[1]]])
    apriori_list += [tuple([at[0] for at in i[1]])]
    index += 1
apriori_list2 = []
for i in apriori_list:
    al2 = []
    for j in i:
        if j.upper() in list_of_aspects:
            al2 += [j.upper()]
    apriori_list2 += [tuple(al2)]

columnsi = [item for item in list(set(list_of_aspects)) if len(item)>1 and '!' not in item and item not in set(stopwords.words('english')) and item != 'IM'][:-2]
mylist = [item for item in list_of_aspects if len(item)>1 and '!' not in item and item not in set(stopwords.words('english')) and item != 'IM'][:-2]
from collections import Counter
c = Counter(mylist)
cp = c.most_common(6)
cp = [ap[0] for ap in cp]
columnsi = cp

real_df = pd.DataFrame('nan', index=list(range(0,len(rev1['Text']))), columns=columnsi, dtype = object)
for i in aspectListtuple:
    if i[0] in cp:
        real_df.at[i[1], i[0]] = i[0]
# real_df = real_df.replace(np.nan, 0)
# real_df = real_df.replace('', np.nan)
real_list = real_df.values.tolist()
real_list.insert(0,real_df.columns.values.tolist())
apriori_real_list = []
for i in real_list:
    if i != ['nan','nan','nan','nan','nan','nan']:
        apriori_real_list += [i]
association_results = list(apriori(apriori_real_list, min_support=0.01, min_confidence=0.7, min_lift=0.5, min_length=2))
print(association_results)

[RelationRecord(items=frozenset({'nan'}), support=0.9937888198757764, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=0.9937888198757764, lift=1.0)]), RelationRecord(items=frozenset({'ACER', 'nan'}), support=0.16149068322981366, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ACER'}), items_add=frozenset({'nan'}), confidence=0.9629629629629629, lift=0.9689814814814814)]), RelationRecord(items=frozenset({'nan', 'COMPUTER'}), support=0.13043478260869565, ordered_statistics=[OrderedStatistic(items_base=frozenset({'COMPUTER'}), items_add=frozenset({'nan'}), confidence=0.9545454545454545, lift=0.9605113636363636)]), RelationRecord(items=frozenset({'nan', 'MONITOR'}), support=0.4161490683229814, ordered_statistics=[OrderedStatistic(items_base=frozenset({'MONITOR'}), items_add=frozenset({'nan'}), confidence=0.9852941176470588, lift=0.9914522058823529)]), RelationRecord(items=frozenset({'PRICE', 'nan'}), support=0.17391304347826

As is shown in this file since top 6 aspect counts are: ('MONITOR', 72), ('PRICE', 28), ('ACER', 26), ('TIME', 24), ('COMPUTER', 22), ('SCREEN', 17) these tags are not sufficiently spread between the tagged rows in order to create very good rules for this file specifically. The appriori algorithm may not be the approach to take for this file as the data is not fully spread out. Even when using a minimum support of 1% as suggested by Minqing Hu and Bing Liu: Mining and Summarizing Customer Reviews.